In [1]:
import pandas as pd

In [4]:
# load dataset
data_dir = '../data/google_trace_timeseries/'
data_name = {'3': 'data_resource_usage_3Minutes_6176858948.csv',
             '5': 'data_resource_usage_5Minutes_6176858948.csv',
             '8': 'data_resource_usage_8Minutes_6176858948.csv',
             '10': 'data_resource_usage_10Minutes_6176858948.csv'}
col_names = ['time_stamp', 'numberOfTaskIndex', 'numberOfMachineId', 
             'meanCPUUsage', 'canonical_memory_usage', 'AssignMem',
             'unmapped_cache_usage', 'page_cache_usage', 'max_mem_usage',
             'mean_diskIO_time', 'mean_local_disk_space', 'max_cpu_usage',
             'max_disk_io_time', 'cpi', 'mai', 'sampling_portion',
             'agg_type', 'sampled_cpu_usage']

dataset_3 = pd.read_csv(data_dir+data_name['3'], header=None, names=col_names)
dataset_3.head()

,time_stamp,numberOfTaskIndex,numberOfMachineId,meanCPUUsage,canonical_memory_usage,AssignMem,unmapped_cache_usage,page_cache_usage,max_mem_usage,mean_diskIO_time,mean_local_disk_space,max_cpu_usage,max_disk_io_time,cpi,mai,sampling_portion,agg_type,sampled_cpu_usage
0,600.0,3089.0,3089.0,0.668381,0.158746,0.351781,0.018816,0.030655,0.189231,0.006224,0.004191,5.33933,0.761954,182.528000,0.388110,0.0,3089.0,0.0
1,780.0,3090.0,3090.0,0.668381,0.158746,0.351781,0.018816,0.030655,0.189231,0.006224,0.004191,5.33933,0.761954,182.528000,0.388110,0.0,3090.0,0.0
2,960.0,3089.0,3089.0,0.739129,0.164337,0.360930,0.019065,0.032660,0.196537,0.002072,0.004189,5.17264,0.336054,187.960999,0.396634,0.0,3089.0,0.0
3,1140.0,3089.0,3089.0,0.739129,0.164337,0.360930,0.019065,0.032660,0.196537,0.002072,0.004189,5.17264,0.336054,187.960999,0.396634,0.0,3089.0,0.0
4,1320.0,3089.0,3089.0,0.558318,0.164920,0.362985,0.019132,0.033352,0.189972,0.000705,0.004188,4.69266,0.093905,194.289001,0.424709,0.0,3089.0,0.0


In [5]:
dataset_3_compact = dataset_3.loc[:, ('meanCPUUsage', 'canonical_memory_usage', 'mean_diskIO_time', 'mean_local_disk_space')]
dataset_3_compact.plot()

In [9]:
data_cpu = dataset_3_compact.loc[:, 'meanCPUUsage']
data_cpu.to_csv('cpu.csv', index=None)
data_cpu = dataset_3_compact.loc[:,'meanCPUUsage'].values
print(data_cpu.shape)
data_cpu = data_cpu.reshape(data_cpu.shape[0],1)
print(data_cpu.shape)

(13920,)
(13920, 1)


In [5]:
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

 
data = series_to_supervised(data_cpu, 1, 1)
print(data.head())

   var1(t-1)   var1(t)
1   0.668381  0.668381
2   0.668381  0.739129
3   0.739129  0.739129
4   0.739129  0.558318
5   0.558318  0.539101


In [1]:
import tensorflow as tf

In [2]:
# create the model
batch_size = 50
num_steps = 4

hidden_size = 100
num_layers = 2
dropout = 0.5

# create LSTM network
# set up the state storage / extraction
init_state = tf.placeholder(tf.float32, [num_layers, 2, batch_size, hidden_size])
state_per_layer_list = tf.unstack(init_state, axis=0)
rnn_tuple_state = tuple(
    [tf.contrib.rnn.LSTMStateTuple(state_per_layer_list[idx][0], 
                                   state_per_layer_list[idx][1]) 
     for idx in range(num_layers)]
)

# create an LSTM cell to be unrolled
cell = tf.contrib.rnn.LSTMCell(hidden_size, forget_bias=1.0)
# add a dropout wrapper
cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=dropout)
# many layers
layers = tf.contrib.rnn.MultiRNNCell([cell for _ in range(num_layers)], state_is_tuple=True)

# crate dynamic RNN object
output, state = tf.nn.dynamic_rnn(layers, inputs, dtype=tf.float32, initial_state=rnn_tuple_state)

NameError: name 'inputs' is not defined